In [8]:
import os, fiona, pandas as pd, geopandas as gpd, requests
from bs4 import BeautifulSoup

In [9]:
folder_path = 'IGM_Data'

# Liste per tenere traccia dei percorsi dei file .gdb e dei layer unici
gdb_path_list = []
unique_layers = set()

# Trova tutti i file .gdb nella cartella e i loro layer unici
for root, dirs, files in os.walk(folder_path):
    for dir in dirs:
        if dir.endswith('.gdb'):
            gdb_path = os.path.join(root, dir)
            gdb_path_list.append(gdb_path)  # Aggiungi il percorso alla lista dei percorsi .gdb
            try:
                layers = fiona.listlayers(gdb_path)
                unique_layers.update(layers)  # Aggiungi i layer al set dei layer unici
            except Exception as e:
                print(f"Non è stato possibile elencare i layer in {gdb_path}: {e}")

# Converti il set dei layer unici in una lista
unique_layers_list = list(unique_layers)

Non è stato possibile elencare i layer in IGM_Data/BO_dbsn_3J0M0f/Bologna_dbsn.gdb: 'IGM_Data/BO_dbsn_3J0M0f/Bologna_dbsn.gdb' not recognized as a supported file format.
Non è stato possibile elencare i layer in IGM_Data/FC_dbsn_3N3H0u/Forli'-Cesena_dbsn.gdb: 'IGM_Data/FC_dbsn_3N3H0u/Forli'-Cesena_dbsn.gdb' not recognized as a supported file format.
Non è stato possibile elencare i layer in IGM_Data/FE_dbsn_6l2Z6x/Ferrara_dbsn.gdb: 'IGM_Data/FE_dbsn_6l2Z6x/Ferrara_dbsn.gdb' not recognized as a supported file format.
Non è stato possibile elencare i layer in IGM_Data/MO_dbsn_1z8t1l/Modena_dbsn.gdb: 'IGM_Data/MO_dbsn_1z8t1l/Modena_dbsn.gdb' not recognized as a supported file format.
Non è stato possibile elencare i layer in IGM_Data/PC_dbsn_9N1E3y/Piacenza_dbsn.gdb: 'IGM_Data/PC_dbsn_9N1E3y/Piacenza_dbsn.gdb' not recognized as a supported file format.
Non è stato possibile elencare i layer in IGM_Data/PR_dbsn_4J0E3N/Parma_dbsn.gdb: 'IGM_Data/PR_dbsn_4J0E3N/Parma_dbsn.gdb' not recognized 

In [ ]:
# URL della pagina da cui estrarre le informazioni
url = "https://www.igmi.org/dbsn_supporto/dbsn/"

# Scarica il contenuto della pagina
response = requests.get(url)

# Imposta la codifica corretta per la risposta, se conosciuta
response.encoding = 'utf-8'

soup = BeautifulSoup(response.text, 'html.parser')

# Dizionario per conservare le informazioni estratte
layer_info = {}

for layer in unique_layers_list:
    # Cerca il titolo del layer nel contenuto della pagina
    layer_element = soup.find(lambda tag: tag.name == "h5" and layer in tag.text)
    if layer_element:
        # Trova la tabella HTML successiva
        table = layer_element.find_next('table')

        # Se la tabella è trovata, leggila con pandas
        if table:
            df = pd.read_html(str(table), encoding='utf-8')[0]
            df['Layer'] = layer  # Aggiungi il nome del layer
            layer_info[layer] = df

# Unisci tutti i DataFrame in layer_info in un unico DataFrame
all_layers_df = pd.concat(layer_info.values(), ignore_index=True)

# Specifica il percorso locale dove salvare le tabelle
local_path = 'IGM_Data'
file_name = 'tables_IGM_DBSM_description.csv'
file_path = os.path.join(local_path, file_name)

# Controlla se il file esiste già
if not os.path.exists(file_path):
    # Assicurati che la directory esista
    os.makedirs(local_path, exist_ok=True)

    # Salva la tabella unita come file CSV
    all_layers_df.to_csv(file_path, index=False)
    print(f"File salvato come {file_path}")
else:
    # Leggi il file CSV se esiste già
    all_layers_df = pd.read_csv(file_path)
    print(f"Il file {file_name} esiste già in {local_path}")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL della pagina da cui estrarre le informazioni
url = "https://www.igmi.org/dbsn_supporto/dbsn/"

# Scarica il contenuto della pagina
response = requests.get(url)

# Imposta la codifica corretta per la risposta, se conosciuta
response.encoding = 'utf-8'

soup = BeautifulSoup(response.text, 'html.parser')

# Dizionario per conservare le informazioni estratte
layer_info = {}

for layer in unique_layers_list:
    # Cerca il titolo del layer nel contenuto della pagina
    layer_element = soup.find(lambda tag: tag.name == "h5" and layer in tag.text)
    if layer_element:
        # Trova la tabella HTML successiva
        table = layer_element.find_next('table')

        # Se la tabella è trovata, leggila con pandas
        if table:
            df = pd.read_html(str(table), encoding='utf-8')[0]
            df['Layer'] = layer  # Aggiungi il nome del layer
            layer_info[layer] = df

# Unisci tutti i DataFrame in layer_info in un unico DataFrame
all_layers_df = pd.concat(layer_info.values(), ignore_index=True)

In [11]:
# Directory dove salvare i file shapefile estratti
output_shp_directory = 'IGM_Data/shp_extracted'
os.makedirs(output_shp_directory, exist_ok=True)

# Dizionario per accumulare i GeoDataFrame per ciascun layer
gdf_accumulator = {layer: [] for layer in unique_layers_list}

for gdb_path in gdb_path_list:
    layers = fiona.listlayers(gdb_path)
    for layer in layers:
        if layer in unique_layers_list:
            try:
                # Leggi il layer come GeoDataFrame
                gdf = gpd.read_file(gdb_path, layer=layer)

                # Aggiungi il GeoDataFrame all'accumulatore
                gdf_accumulator[layer].append(gdf)
                print(f"Processato {layer} del file {gdb_path}")
            except Exception as e:
                print(f"Errore durante l'elaborazione del layer {layer} nel file {gdb_path}: {e}")

# Ora unisci e salva i GeoDataFrame accumulati per ciascun layer
for layer, gdf_list in gdf_accumulator.items():
    if gdf_list:  # Se c'è almeno un GeoDataFrame per questo layer
        merged_gdf = pd.concat(gdf_list, ignore_index=True)

        # Salva il risultato come shapefile
        output_path = os.path.join(output_shp_directory, f"{layer}.shp")
        merged_gdf.to_file(output_path)
        print(f"Layer {layer} unito e salvato come shapefile in {output_path}")

DriverError: 'IGM_Data/BO_dbsn_3J0M0f/Bologna_dbsn.gdb' not recognized as a supported file format.